In [1]:
import os

base_dir = "/storage"
base_dir_7t = [base_dir + "/HCP_7T/" + i   for i in os.listdir(base_dir + "/HCP_7T") if len(i) == 6]
base_dir_3t = [base_dir + "/HCP_3T/" + i   for i in os.listdir(base_dir + "/HCP_3T") if len(i) == 6]

path_7t = {}
path_3t = {}



for i in base_dir_7t:
    path_7t[i[-6:]] = {"3d_scan" : i + "/T1w/T1w_acpc_dc_restore_1.05.nii.gz" ,"data" : i + "/T1w/Diffusion_7T/data.nii.gz" 
                       , "bvals" : i + "/T1w/Diffusion_7T/bvals" , "bvecs" : i + "/T1w/Diffusion_7T/bvecs"
                      , "brain_mask" : i + "/T1w/Diffusion_7T/nodif_brain_mask.nii.gz"
                      , "grad_dev" : i + "/T1w/Diffusion_7T/grad_dev.nii.gz"}
for i in base_dir_3t:
    path_3t[i[-6:]] = {"3d_scan" : i + "/T1w/T1w_acpc_dc_restore_1.25.nii.gz" , "data" : i + "/T1w/Diffusion/data.nii.gz" 
                       , "bvals" : i + "/T1w/Diffusion/bvals" , "bvecs" : i + "/T1w/Diffusion/bvecs"
                       , "brain_mask" : i + "/T1w/Diffusion/nodif_brain_mask.nii.gz"
                      , "grad_dev" : i + "/T1w/Diffusion/grad_dev.nii.gz"}
    
    
path = {'3T': path_3t, "7T": path_7t}
p = list(path_7t.keys())
q = list(path_3t.keys())
common = list(set(p) & set(q))

print("number of common Subjects ",len(common))

def get_ids():
    return common

def load_hcp(id_load,res,crop = 1):
    load_from = path[res][id_load]
    data , affine = load_nifti(load_from["data"])
    mask,affine = load_nifti(load_from["brain_mask"])
    
    bvals, bvecs = read_bvals_bvecs(load_from['bvals'], load_from['bvecs'])
    gtab = gradient_table(bvals, bvecs)
    
    return data[:,:,crop:-crop,:],mask[:,:,crop:-crop],gtab

number of common Subjects  171


In [2]:
from dipy.io.image import load_nifti
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
import numpy as np
import os
import matplotlib.pyplot as plt
from numpy.linalg import eig
from numpy.linalg import inv
from numpy.linalg import pinv
# from numpy.linalg import lstsq
from scipy.linalg import lstsq
from numpy.linalg import solve
from numpy import inf
import numpy as np


dsm = np.array([0.91, 0.416, 0,0, 0.91, 0.416,0.416, 0, 0.91,0.91, -0.416, 0,0, 0.91, -0.416,-0.416, 0, 0.91])
dsm = dsm.reshape(6,3)
dsm_norm = np.copy(dsm)
dsm_mag = np.sqrt(dsm[:,0]**2 + dsm[:,1]**2 + dsm[:,2]**2)
for i in range(3):
    dsm_norm[:,i] = dsm[:,i] / dsm_mag

def rot3d(arg):
    x,y,z = arg[0],arg[1],arg[2]
    Rx = np.array([[1 ,0 ,0 ],[0,np.cos(x),-np.sin(x)],[0 ,np.sin(x) ,np.cos(x)]])
    Ry = np.array([[np.cos(y),0 ,np.sin(y) ],[0,1,0],[-np.sin(y),0,np.cos(y)]])
    Rz = np.array([[np.cos(z) ,-np.sin(z) ,0 ],[np.sin(z),np.cos(z),0],[0 ,0,1]])
    R = Rx @ Ry @ Rz
    return R

def amatrix(mat):
    
    a = [mat[:,0] * mat[:,0],2 * mat[:,0] * mat[:,1], 2* mat[:,0] * mat[:,2],
        mat[:,1] * mat[:,1],2 * mat[:,1] * mat[:,2], mat[:,2] * mat[:,2]]
    return np.array(a).T

def optimal_dirs(gtab,num_iter,num_dirs,debug = False,base_bval = 5):
    rotang_all = []
    angerr_all  = []
    condnum_all = []
    ind_all = []
    dirs = np.array(gtab.bvecs[np.where(gtab.bvals != base_bval)[0]])
    for i in range(0,num_iter):
        
        d = np.random.rand(1,3) * 2 * np.pi
        rotang = d[0]
        R = rot3d(rotang)
        dsm_rot = (rot3d(d[0]) @ dsm_norm.T).T
        
        ang_error = np.degrees(np.arccos(abs(dsm_rot @ dirs.T)))
        minerrors,idx = np.amin(ang_error,1),np.argmin(ang_error,1)

        mean_ang_err = np.mean(np.amin(ang_error,1))
        condnum = np.linalg.cond(amatrix(dirs[idx]))
        
        idx.sort()
        if (mean_ang_err < 5 and condnum < 1.6):
            if ((len(ind_all) == 0 ) or  len(np.where((ind_all == idx).all(axis=1))[0]) == 0 ):
                angerr_all.append(mean_ang_err)
                condnum_all.append(condnum)
                ind_all.append(idx)
                rotang_all.append(rotang)
    condnum_all = np.array(condnum_all)
    indx  = condnum_all.argsort()[:num_dirs]
    if (debug):
        print("Lowest Condition Number : ",condnum_all[indx])
    ind_use = np.array(ind_all)[indx]
    condnum_use = condnum_all[condnum_all.argsort()[:5]]
    angerr_use = np.array(angerr_all)[indx]
    rotang_use = np.array(rotang_all)[indx]
    return ind_use,condnum_all[indx]


In [3]:
from dipy.segment.mask import median_otsu
from dipy.reconst.dti import fractional_anisotropy, color_fa
import dipy.reconst.dti as dti
import h5py

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
for i in common:
    print("Current ID",i)
    data,mask,gtab = load_hcp(i,'3T')
    name = "/storage/HCP_3T/" + i + "/" + i + ".h5"
    hf = h5py.File(name, 'w')
    temp = data[:,:,:,gtab.bvals>5]
    idx,cond= optimal_dirs(gtab,10000,5,debug = True,base_bval = int(min(gtab.bvals)))
    base_bvals_vols = data[:,:,:,gtab.bvals==int(min(gtab.bvals))]
    base_bvals = gtab.bvecs[gtab.bvals==int(min(gtab.bvals)),:],gtab.bvals[gtab.bvals==int(min(gtab.bvals))]
    dw_bvals = gtab.bvecs[gtab.bvals!=int(min(gtab.bvals)),:],gtab.bvals[gtab.bvals!=int(min(gtab.bvals))]
    for j in range(3):
        sample = np.random.randint(0,base_bvals_vols.shape[3])
        b0 = base_bvals_vols[:,:,:,sample]
        bvals6 = np.insert(dw_bvals[1][idx[j]],0,base_bvals[1][sample])
        bvecs6 = np.insert(dw_bvals[0][idx[j]],0,base_bvals[0][sample],axis = 0)
        data6 = temp[:,:,:,idx[j]]
        data6 = np.insert(data6,0,b0,axis =3)
        hf.create_dataset('volumes'+ str(j), data=data6)
        hf.create_dataset('bvals'+ str(j), data=bvals6)
        hf.create_dataset('bvecs'+ str(j), data=bvecs6)
    hf.create_dataset('idx', data=idx)
    hf.create_dataset('cond', data=cond)
    hf.create_dataset('mask', data=mask)
    
    hf.close()
    print(name,"Done")
    
    name = "/storage/HCP_3T/" + i + "/" + i + "_GT.h5"
    hf = h5py.File(name, 'w')
    
    tenmodel = dti.TensorModel(gtab)
    tenfit = tenmodel.fit(data,mask=mask)
    tensor_vals = dti.lower_triangular(tenfit.quadratic_form)
    FA = fractional_anisotropy(tenfit.evals)
    MD = dti.mean_diffusivity(tenfit.evals)
    FA[np.isnan(FA)] = 0
    FA = np.clip(FA, 0, 1)
    RGB = color_fa(FA, tenfit.evecs)
    
    hf.create_dataset('tensor_vals', data=tensor_vals)
    hf.create_dataset('ADC', data=MD)
    hf.create_dataset('FA', data=FA)
    hf.create_dataset('color_FA', data=RGB)
    
    hf.close()
    print(name,"Done")

Current ID 105923


/tmp/ipykernel_5038/4007092059.py:51: RuntimeWarning: invalid value encountered in arccos
  ang_error = np.degrees(np.arccos(abs(dsm_rot @ dirs.T)))


Lowest Condition Number :  [1.42135102 1.42417912 1.43586124 1.43906316 1.44036295]
/storage/HCP_3T/105923/105923.h5 Done


In [ ]:
%who

In [ ]:
common[0]

In [ ]:
data,mask,gtab = load_hcp(common[0],'3T')

In [ ]:
idx = optimal_dirs(gtab,10000,5,debug = True,base_bval = 5)

In [ ]:
gtab.bvals[idx[0]],gtab.bvecs[idx[0]]

In [ ]:
bvals6,bvecs6 = np.insert(gtab.bvals[idx[0]],0,gtab.bvals[0]),np.insert(gtab.bvecs[idx[0]],0,gtab.bvecs[0],axis = 0)

In [ ]:
bvals6,bvecs6

In [ ]:
len(bvals6),len(bvecs6)

In [ ]:
gtab6 = gradient_table(bvals6, bvecs6)

In [ ]:
temp = data[:,:,:,gtab.bvals>5]

In [ ]:
data6 = temp[:,:,:,idx[0]]

In [ ]:
data6.shape

In [ ]:
data6 = np.insert(data6,0,data[:,:,:,0],axis =3)

In [ ]:
data6.shape

In [ ]:
import h5py
name = "/storage/HCP_3T/" + common[0] + "/" + common[0] + ".h5"

In [ ]:
name

In [ ]:

hf = h5py.File(name, 'w')

In [ ]:
hf.create_dataset('volumes', data=data6)
hf.create_dataset('bvals', data=bvals6)
hf.create_dataset('bvecs', data=bvecs6)

In [ ]:
hf.close()

In [ ]:
name = "/storage/HCP_3T/" + common[0] + "/" + common[0] + ".h5"
hf = h5py.File(name, 'r')

In [ ]:
list(hf.keys())

In [ ]:
for i in list(hf.keys())[3:6]:
    print(i,np.array(hf.get(i)))

In [ ]:
np.array(hf.get('volumes0')).shape

In [ ]:
for i in list(hf.keys())[:3]:
    print(i,np.array(hf.get(i)))

In [ ]:
np.array(hf.get('cond'))

In [ ]:
np.array(hf.get('idx'))

In [ ]:
for i in list(hf.keys()):
    print(i,np.array(hf.get(i)).shape)

In [ ]:
hf.close()

In [ ]:
common[0]

In [ ]:
name = "/storage/HCP_3T/" + common[0] + "/" + common[0] + "_proc.h5"
hf = h5py.File(name, 'r')

In [ ]:
np.array(hf.get(i)).shape

In [ ]:
np.array(hf.get('bvals1'))

In [ ]:
for i in list(hf.keys()):
    print(i,np.array(hf.get(i)).shape)

In [ ]:
np.array(hf.get('ADC'))

In [ ]:
i=70
f, axis = plt.subplots(1, 3)
f.set_figwidth(20)
f.set_figheight(10)
axis[0].set_title('FA "fractional anisotropy"')
axis[0].imshow(np.array(hf.get('FA'))[:,:,i])
axis[1].set_title('MD "Mean Diffusivity or Average Diffusion Coefficent(ADC)"')
axis[1].imshow(np.array(hf.get('ADC'))[:,:,i])
axis[2].set_title('MD_RGB "Principal Diffusion Directions"')
axis[2].imshow(np.array(hf.get('color_FA'))[:,:,i])